# Resumen del Capítulo: Ingeniería de Características a partir de Texto

## Introducción al Capítulo

En muchos conjuntos de datos, la información puede provenir de campos de texto libre,
como descripciones de incidentes o reseñas de clientes. A diferencia de los datos tabulares,
el texto varía en longitud, contenido y estilo de escritura. El objetivo de este capítulo
es mostrar cómo transformar esta información textual en características predictivas numéricas
que pueden ser utilizadas en modelos de machine learning.

Las técnicas cubiertas pertenecen al campo del **Procesamiento del Lenguaje Natural (NLP)**,
que se ocupa de programar computadoras para comprender el lenguaje humano. En concreto,
el capítulo se enfoca en métodos para extraer rápidamente características de textos cortos,
capturando su complejidad a través de parámetros estadísticos (como la longitud de las palabras,
el número de palabras únicas y el conteo de oraciones).

## Librerías y Requisitos Técnicos

| Librería           | Propósito                                                                                     |
|--------------------|-----------------------------------------------------------------------------------------------|
| **pandas**         | Manipulación de datos y funciones vectorizadas de procesamiento de strings (str).            |
| **scikit-learn**   | Carga de conjuntos de datos (e.g., 20 Newsgroup) y transformers clave.                      |
| **NLTK**           | Herramienta integral de Python para NLP, esencial para la tokenización y stemming.          |

In [1]:
# ============================================================================
# IMPORTS - Librerías y Módulos Necesarios
# ============================================================================

# --- Sistema Operativo y Rutas ---
import os  # Operaciones del sistema operativo (no usado directamente, pero disponible)
from pathlib import (
    Path,
)

# --- Manipulación de Datos ---
import pandas as pd  # Análisis y manipulación de datos en DataFrames
# Usado para: crear tablas, aplicar operaciones vectorizadas en texto

# --- Natural Language Toolkit (NLTK) - Core ---
import nltk  # Librería principal de NLP para Python
# Usado para: descargar recursos y configurar rutas de datos

# --- NLTK - Tokenización ---
from nltk.tokenize import sent_tokenize  # Divide texto en oraciones individuales
# Usado en: Receta 2 para contar número de oraciones

# --- NLTK - Stop Words ---
from nltk.corpus import (
    stopwords,
)  # Acceso a listas de palabras comunes sin valor semántico
# Usado en: Receta 5 para filtrar palabras como 'the', 'a', 'is'

# --- NLTK - Stemming ---
from nltk.stem.snowball import SnowballStemmer  # Reduce palabras a su raíz/base
# Usado en: Receta 5 para convertir 'running', 'runs' -> 'run'

# --- Scikit-learn - Datasets ---
from sklearn.datasets import (
    fetch_20newsgroups,
)  # Descarga dataset de grupos de noticias
# Usado para: obtener textos de ejemplo para análisis

# --- Scikit-learn - Vectorización de Texto ---
from sklearn.feature_extraction.text import (
    CountVectorizer,
)  # Convierte texto a matriz Bag-of-Words
# Usado en: Receta 3 para contar frecuencia de palabras

from sklearn.feature_extraction.text import (
    TfidfVectorizer,
)  # Convierte texto a matriz TF-IDF
# Usado en: Receta 4 para ponderar importancia de palabras

# Configuración de directorios del proyecto
PROJECT_ROOT = Path.cwd()
NLTK_DATA_DIR = PROJECT_ROOT / "nltk_data"
SKLEARN_DATA_DIR = PROJECT_ROOT / "sklearn_data"

# Crear directorios si no existen
NLTK_DATA_DIR.mkdir(exist_ok=True)
SKLEARN_DATA_DIR.mkdir(exist_ok=True)

# Configurar NLTK para usar el directorio del proyecto
nltk.data.path.insert(0, str(NLTK_DATA_DIR))

# Descargar recursos de NLTK en el directorio del proyecto
print("Descargando recursos de NLTK...")
nltk.download("punkt", download_dir=str(NLTK_DATA_DIR), quiet=True)
nltk.download("stopwords", download_dir=str(NLTK_DATA_DIR), quiet=True)
nltk.download("punkt_tab", download_dir=str(NLTK_DATA_DIR), quiet=True)
print("✓ Recursos de NLTK descargados\n")

Descargando recursos de NLTK...
✓ Recursos de NLTK descargados



In [2]:
# Función auxiliar para cargar datos en el directorio del proyecto
def load_newsgroups_data(subset="train"):
    """
    Carga el dataset 20 Newsgroups en el directorio del proyecto.

    Parameters:
        subset (str): 'train' o 'test'

    Returns:
        pd.DataFrame: DataFrame con columna 'text'
    """
    print(f"Cargando dataset 20 Newsgroups ({subset})...")
    data = fetch_20newsgroups(
        subset=subset,
        data_home=str(SKLEARN_DATA_DIR),
        remove=("headers", "footers", "quotes"),  # Limpieza inicial
    )
    df = pd.DataFrame(data.data, columns=["text"])
    print(f"✓ Dataset cargado: {len(df)} documentos\n")
    return df

### Verificación del dataset original y de los archivos descargados.

Antes de comenzar con las recetas, vemos un ejemplo del dataset descargado sin procesar:

In [3]:
df = load_newsgroups_data(subset="train")
df.head()

Cargando dataset 20 Newsgroups (train)...
✓ Dataset cargado: 11314 documentos



,text
0,I was wondering if anyone out there could enli...
1,A fair number of brave souls who upgraded thei...
2,"well folks, my mac plus finally gave up the gh..."
3,\nDo you have Weitek's address/phone number? ...
4,"From article <C5owCB.n3p@world.std.com>, by to..."


In [4]:
print("\n" + "=" * 70)
print("VERIFICACIÓN DE ARCHIVOS DEL PROYECTO")
print("=" * 70)

print(f"\nDirectorio del proyecto: {PROJECT_ROOT}")
print(f"\nArchivos NLTK: {NLTK_DATA_DIR}")
if NLTK_DATA_DIR.exists():
    nltk_files = list(NLTK_DATA_DIR.rglob("*"))
    print(f" ✓ {len(nltk_files)} archivos encontrados")

print(f"\nArchivos Scikit-learn: {SKLEARN_DATA_DIR}")
if SKLEARN_DATA_DIR.exists():
    sklearn_files = list(SKLEARN_DATA_DIR.rglob("*"))
    print(f" ✓ {len(sklearn_files)} archivos encontrados")

print("\n ✓ Todos los archivos están en el directorio del proyecto")


VERIFICACIÓN DE ARCHIVOS DEL PROYECTO

Directorio del proyecto: /home/juani/Documentos/Facultad/Ciencia de Datos/proyectos/lab3/Faith No More

Archivos NLTK: /home/juani/Documentos/Facultad/Ciencia de Datos/proyectos/lab3/Faith No More/nltk_data
 ✓ 179 archivos encontrados

Archivos Scikit-learn: /home/juani/Documentos/Facultad/Ciencia de Datos/proyectos/lab3/Faith No More/sklearn_data
 ✓ 1 archivos encontrados

 ✓ Todos los archivos están en el directorio del proyecto


# Recetas Clave del Capítulo

El capítulo se estructura alrededor de cinco recetas principales, que transforman 
el texto sin procesar en datos estructurados y numéricos.

## **Receta 1:** Conteo de Caracteres, Palabras y Vocabulario

Esta receta se centra en medir la complejidad del texto a través de estadísticas básicas. 
Las descripciones más largas y ricas en vocabulario único suelen contener más información.

Características extraídas (usando pandas):
1. **Número total de caracteres:** Incluye letras, números, símbolos y espacios.
2. **Número total de palabras**.
3. **Número total de palabras únicas (vocabulario)**.
4. **Diversidad léxica:** Cociente entre el número total de palabras y el número de palabras únicas.
5. **Longitud promedio de la palabra:** Cociente entre el número de caracteres y el número de palabras.

In [5]:
print("=" * 70)
print("RECETA 1: Conteo de Caracteres, Palabras y Vocabulario")
print("=" * 70)

# Cargar datos
df = load_newsgroups_data(subset="train")

# Conteo de caracteres (después de strip para eliminar espacios en blanco)
df["num_char"] = df["text"].str.strip().str.len()

# Conteo de palabras (split() divide el texto en espacios en blanco)
df["num_words"] = df["text"].str.split().str.len()

# Conteo de vocabulario (palabras únicas)
# Usar lower() para evitar que 'Palabra' y 'palabra' sean tratadas como diferentes
df["num_vocab"] = df["text"].str.lower().str.split().apply(lambda x: len(set(x)))

# Diversidad Léxica (evitar división por cero)
df["lexical_div"] = df["num_words"] / df["num_vocab"].replace(0, 1)

# Longitud Promedio de Palabras (evitar división por cero)
df["ave_word_length"] = df["num_char"] / df["num_words"].replace(0, 1)

print("\nEstadísticas básicas del texto:")
df[["num_char", "num_words", "num_vocab", "lexical_div", "ave_word_length"]].describe()

RECETA 1: Conteo de Caracteres, Palabras y Vocabulario
Cargando dataset 20 Newsgroups (train)...
✓ Dataset cargado: 11314 documentos


Estadísticas básicas del texto:


,num_char,num_words,num_vocab,lexical_div,ave_word_length
count,11314.000000,11314.000000,11314.000000,11314.000000,11314.000000
mean,1216.228566,185.827382,106.218844,1.322594,5.912501
std,4038.305818,523.971647,183.974420,0.395230,2.615407
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,235.000000,40.000000,35.000000,1.131579,5.438596
50%,490.000000,83.000000,65.000000,1.280702,5.761905
75%,982.750000,167.000000,114.000000,1.475588,6.176785
max,74878.000000,11765.000000,3480.000000,6.438086,78.000000


In [6]:
print("\nPrimeras 5 filas:")
df[["text", "num_char", "num_words", "num_vocab"]].head()


Primeras 5 filas:


,text,num_char,num_words,num_vocab
0,I was wondering if anyone out there could enli...,475,91,67
1,A fair number of brave souls who upgraded thei...,530,90,74
2,"well folks, my mac plus finally gave up the gh...",1659,307,195
3,\nDo you have Weitek's address/phone number? ...,93,15,15
4,"From article <C5owCB.n3p@world.std.com>, by to...",448,72,61


## **Receta 2:** Estimación de la Complejidad por Conteo de Oraciones

Capturar el número de oraciones ofrece información sobre la cantidad de contenido en el texto, 
ya que las descripciones con múltiples oraciones tienden a ser más informativas. 
Este proceso se denomina **tokenización de oraciones**.

**Nota importante:** La tokenización de oraciones se basa en la puntuación y la capitalización. 
Si planea contar oraciones, este paso debe realizarse antes de cualquier eliminación de 
puntuación o cambio de caso.

In [7]:
print("\n" + "=" * 70)
print("RECETA 2: Conteo de Oraciones")
print("=" * 70)

# Recargar datos limpios (sin headers/footers/quotes)
df_sentences = load_newsgroups_data(subset="train")

# Función robusta para contar oraciones
def count_sentences(text):
    """Cuenta oraciones manejando textos vacíos."""
    if pd.isna(text) or not text.strip():
        return 0
    try:
        return len(sent_tokenize(text))
    except Exception as e:
        print(f"Error al tokenizar: {e}")
        return 0


# Crear característica de número de oraciones
df_sentences["num_sent"] = df_sentences["text"].apply(count_sentences)

print("\nEstadísticas de número de oraciones:")
df_sentences[["num_sent"]].describe()


RECETA 2: Conteo de Oraciones
Cargando dataset 20 Newsgroups (train)...
✓ Dataset cargado: 11314 documentos


Estadísticas de número de oraciones:


,num_sent
count,11314.000000
mean,11.352130
std,31.888797
min,0.000000
25%,3.000000
50%,6.000000
75%,10.000000
max,921.000000


In [8]:
print("\nEjemplos:")
df_sentences[["text", "num_sent"]].head(3)


Ejemplos:


,text,num_sent
0,I was wondering if anyone out there could enli...,7
1,A fair number of brave souls who upgraded thei...,5
2,"well folks, my mac plus finally gave up the gh...",8


## **Receta 3:** Creación de Características con Bag-of-Words y N-grams

El **Bag-of-Words (BoW)** es una representación simplificada donde cada palabra única 
se convierte en una variable, y su valor representa la frecuencia con la que aparece en el texto. 
El BoW captura la multiplicidad de palabras, pero no su orden o gramática.

Para capturar algo de sintaxis, se usan N-grams, que son secuencias contiguas de n ítems 
(por ejemplo, 2-grams: "Dogs like", "like cats").

In [9]:
print("\n" + "=" * 70)
print("RECETA 3: Bag-of-Words")
print("=" * 70)

# Recargar datos
df_bow = load_newsgroups_data(subset="train")

# Limpieza preliminar: Eliminar puntuación y números
# Reemplazar con espacio para evitar unir palabras
df_bow["text_clean"] = (
    df_bow["text"]
    .str.replace(r"[^\w\s]", " ", regex=True)  # Puntuación -> espacio
    .str.replace(r"\d+", " ", regex=True)  # Números -> espacio
    .str.replace(r"\s+", " ", regex=True)  # Múltiples espacios -> uno solo
    .str.strip()
)

# Configuración de CountVectorizer
vectorizer = CountVectorizer(
    lowercase=True,
    stop_words="english",
    ngram_range=(1, 1),  # Solo unigramas (palabras simples)
    min_df=0.05,  # Frecuencia mínima del 5%
)

# Ajuste y transformación
X = vectorizer.fit_transform(df_bow["text_clean"])

# Captura del BoW en un DataFrame
bagofwords = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

print(
    f"\nDimensiones de la matriz BoW: {bagofwords.shape[0]} filas x {bagofwords.shape[1]} columnas"
)

print(
    f"\nNúmero de features (palabras únicas): {len(vectorizer.get_feature_names_out())}"
)

print("\nPrimeras 5 palabras más frecuentes:")
top_words = bagofwords.sum().sort_values(ascending=False).head()

# Convertimos la serie a un DataFrame con un nombre de columna
top_words_df = top_words.to_frame(name='Frecuencia')

top_words_df


RECETA 3: Bag-of-Words
Cargando dataset 20 Newsgroups (train)...
✓ Dataset cargado: 11314 documentos


Dimensiones de la matriz BoW: 11314 filas x 88 columnas

Número de features (palabras únicas): 88

Primeras 5 palabras más frecuentes:


,Frecuencia
people,4103
like,3964
don,3885
just,3752
know,3487


In [10]:
print("\nPrimeras 3 filas del BoW:")
bagofwords.head(3)


Primeras 3 filas del BoW:


,able,actually,available,believe,best,better,bit,called,case,com,...,used,using,ve,want,way,windows,work,world,year,years
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,1,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0


## **Receta 4:** Implementación de TF-IDF

TF-IDF es una estadística numérica que mide la relevancia de una palabra en un documento 
específico dentro de una colección completa de documentos.

• **Term Frequency (TF):** Simplemente la cuenta de la palabra en un texto individual.
• **Inverse Document Frequency (IDF):** Mide cuán común es la palabra en todos los documentos. 
  Las palabras que aparecen en casi todos los documentos (como 'the' o 'a') tendrán un bajo peso.

TF-IDF pondera la importancia; una palabra es importante si aparece mucho en un texto (tf alto) 
y pocas veces en el resto de los textos (idf alto).

In [11]:
print("\n" + "=" * 70)
print("RECETA 4: TF-IDF")
print("=" * 70)

# Usar los datos ya limpios de la receta anterior
# Configuración de TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    lowercase=True,
    stop_words="english",
    ngram_range=(1, 1),
    min_df=0.05,
)

# Ajuste y transformación
X_tfidf = tfidf_vectorizer.fit_transform(df_bow["text_clean"])

# Captura del TF-IDF en un DataFrame
tfidf_df = pd.DataFrame(
    X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out()
)

print(
    f"\nDimensiones de la matriz TF-IDF: {tfidf_df.shape[0]} filas x {tfidf_df.shape[1]} columnas"
)

print("\nPrimeras 3 filas del TF-IDF (valores normalizados):")
tfidf_df.head(3)


RECETA 4: TF-IDF

Dimensiones de la matriz TF-IDF: 11314 filas x 88 columnas

Primeras 3 filas del TF-IDF (valores normalizados):


,able,actually,available,believe,best,better,bit,called,case,com,...,used,using,ve,want,way,windows,work,world,year,years
0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.412555,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.368366
1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.175511,0.0,0.0,0.0,0.165437,0.174116,0.000000,0.0,0.0,...,0.0,0.0,0.149206,0.0,0.142447,0.0,0.0,0.0,0.0,0.000000


In [12]:
# Comparar con BoW
print("\nComparación BoW vs TF-IDF para la primera fila:")
comparison = pd.DataFrame(
    {"BoW": bagofwords.iloc[0].head(10), "TF-IDF": tfidf_df.iloc[0].head(10)}
)

comparison


Comparación BoW vs TF-IDF para la primera fila:


,BoW,TF-IDF
able,0,0.000000
actually,0,0.000000
available,0,0.000000
believe,0,0.000000
best,0,0.000000
better,0,0.000000
bit,0,0.000000
called,1,0.412555
case,0,0.000000
com,0,0.000000


## **Receta 5:** Limpieza y Stemming de Variables de Texto

La limpieza o preprocesamiento del texto es crucial antes de crear características 
(como BoW o TF-IDF) para estandarizar el contenido y mejorar la precisión del modelo.

**Pasos de Preprocesamiento:**
1. **Eliminación de Puntuación y Números:** Se eliminan caracteres que no son letras o espacios.
2. **Configuración de Caso (Lowercase):** Se establece todo el texto en minúsculas.
3. **Eliminación de Stop Words:** Se remueven palabras comunes y funcionales.
4. **Stemming:** Se reduce cada palabra a su raíz o base.

In [13]:
print("\n" + "=" * 70)
print("RECETA 5: Limpieza y Stemming Completo")
print("=" * 70)

# Recargar datos
df_clean = load_newsgroups_data(subset="train")

# Seleccionar un texto de ejemplo (el primero del dataset)
ejemplo_original = df_clean.iloc[0]["text"]
print("\nTEXTO ORIGINAL:")
print(f"{ejemplo_original}\n")

# Paso 1: Eliminación de puntuación
ejemplo_paso1 = ejemplo_original
ejemplo_paso1 = ejemplo_paso1.replace(r"[^\w\s]", " ")
# Usar re.sub para que funcione correctamente
import re
ejemplo_paso1 = re.sub(r"[^\w\s]", " ", ejemplo_original)

print("PASO 1: Eliminación de puntuación")
print(f"{ejemplo_paso1}\n")

# Paso 2: Eliminación de números
ejemplo_paso2 = re.sub(r"\d+", " ", ejemplo_paso1)
print("PASO 2: Eliminación de números")
print(f"{ejemplo_paso2}\n")

# Paso 3: Conversión a minúsculas
ejemplo_paso3 = ejemplo_paso2.lower()
print("PASO 3: Conversión a minúsculas")
print(f"{ejemplo_paso3}\n")

# Paso 4: Normalizar espacios múltiples
ejemplo_paso4 = re.sub(r"\s+", " ", ejemplo_paso3).strip()
print("PASO 4: Normalización de espacios")
print(f"{ejemplo_paso4}\n")

# Paso 5: Eliminación de Stop Words
STOP_WORDS = set(stopwords.words("english"))

def remove_stopwords(text):
    """Elimina stop words de manera eficiente."""
    if pd.isna(text) or not text.strip():
        return ""
    words = [word for word in text.split() if word not in STOP_WORDS]
    return " ".join(words)

ejemplo_paso5 = remove_stopwords(ejemplo_paso4)
print("PASO 5: Eliminación de Stop Words")
print(f"{ejemplo_paso5}\n")

# Paso 6: Stemming
STEMMER = SnowballStemmer("english")

def stem_words(text):
    """Aplica stemming a cada palabra."""
    if pd.isna(text) or not text.strip():
        return ""
    words = [STEMMER.stem(word) for word in text.split()]
    return " ".join(words)

ejemplo_paso6 = stem_words(ejemplo_paso5)
print("PASO 6: Stemming (reducción a raíz)")
print(f"{ejemplo_paso6}\n")

# Comparación visual final
print("-" * 70)
print("COMPARACIÓN ANTES/DESPUÉS:")
print("-" * 70)
print(f"\n✗ ANTES ({len(ejemplo_original.split())} palabras):")
print(f"{ejemplo_original}\n")
print(f"✓ DESPUÉS ({len(ejemplo_paso6.split())} palabras):")
print(f"{ejemplo_paso6}\n")

# ============================================================================
# PROCESAMIENTO COMPLETO DEL DATASET
# ============================================================================
print("=" * 70)
print("APLICANDO PIPELINE COMPLETO AL DATASET")
print("=" * 70)

# Paso 1: Eliminación de puntuación (reemplazar con espacio)
df_clean["text"] = df_clean["text"].str.replace(r"[^\w\s]", " ", regex=True)

# Paso 2: Eliminación de números
df_clean["text"] = df_clean["text"].str.replace(r"\d+", " ", regex=True)

# Paso 3: Conversión a minúsculas
df_clean["text"] = df_clean["text"].str.lower()

# Paso 4: Normalizar espacios múltiples
df_clean["text"] = df_clean["text"].str.replace(r"\s+", " ", regex=True).str.strip()

# Paso 5: Eliminación de Stop Words
df_clean["text"] = df_clean["text"].apply(remove_stopwords)

# Paso 6: Stemming
df_clean["text"] = df_clean["text"].apply(stem_words)

print(f"\n✓ {len(df_clean)} documentos procesados exitosamente")
print(f"✓ Todos los textos están limpios, sin stop words y con stemming aplicado")
print(f"✓ Datos listos para feature extraction (BoW, TF-IDF, etc.)\n")


RECETA 5: Limpieza y Stemming Completo
Cargando dataset 20 Newsgroups (train)...
✓ Dataset cargado: 11314 documentos


TEXTO ORIGINAL:
I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

PASO 1: Eliminación de puntuación
I was wondering if anyone out there could enlighten me on this car I saw
the other day  It was a 2 door sports car  looked to be from the late 60s 
early 70s  It was called a Bricklin  The doors were really small  In addition 
the front bumper was separate from the rest of the body  This is 
all I know  If anyone can tellme a model name  engine specs 

# Resumen del Pipeline Completo

El proceso de feature engineering en texto es similar a un chef que prepara ingredientes 
para un plato complejo. Inicialmente, tenemos el texto crudo (ingredientes sin procesar). 
La limpieza (Receta 5) es como pelar y cortar los vegetales (eliminar puntuación, stop words 
y encontrar la raíz de la palabra) para que sean útiles. Luego, Recetas 1 y 2 miden la cantidad 
general y el tamaño (¿Cuántos ingredientes hay? ¿Cuántas porciones?). Finalmente, BoW y TF-IDF 
(Recetas 3 y 4) son como catalogar y ponderar la importancia de cada ingrediente: BoW cuenta 
cuántas veces se usa el ajo (frecuencia simple), mientras que TF-IDF determina qué tan esencial 
es el azafrán (un ingrediente raro y específico) en esta receta particular en comparación con 
todas las demás recetas en el libro (la colección de documentos).